In [23]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import torch
from torch import nn
import torch.nn.functional as F
import re

In [4]:
jobs = pd.read_parquet("./all_jobs_db.parquet")
jobs.head(3)

,job_name,company,location,joburl
0,Machine Learning Engineer II,Atlassian,"Bengaluru, Karnataka, India (Remote)",https://www.linkedin.com/jobs/view/4219958889/...
1,Senior Machine Learning Engineer - II,Adobe,"Noida, Uttar Pradesh, India",https://www.linkedin.com/jobs/view/4223614905/...
2,Senior Machine Learning Engineer,Target,"Bengaluru, Karnataka, India (Hybrid)",https://www.linkedin.com/jobs/view/4215587397/...


In [32]:
job_names = jobs['job_name'].values

In [33]:
job_names[:4]

array(['Machine Learning Engineer II',
       'Senior Machine Learning Engineer - II',
       'Senior Machine Learning Engineer', 'Senior ML Engineer'],
      dtype=object)

In [34]:
job_names = [re.sub('[^a-zA-Z ]+', '', x.lower()) for x in job_names]

In [35]:
job_names[:10]

['machine learning engineer ii',
 'senior machine learning engineer  ii',
 'senior machine learning engineer',
 'senior ml engineer',
 'machine learning engineer t',
 'machine learning engineer',
 'sr ml engineer',
 'data insights and analytics engineer',
 'machine learning engineer iii recommender systems',
 'senior machine learning engineer']

In [36]:
s = sorted(list(set(''.join(job_names))))
stoi = {x:i+1 for i,x in enumerate(s)}
itos = {i+1:x for i,x in enumerate(s)}
stoi['.'] = 0
itos[0] = '.'
stoi

{' ': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27,
 '.': 0}

In [61]:
block_size = 3
X, Y = [], []
for w in job_names[:5]:
    print(w)
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context),'--->',itos[ix])
        context = context[1:]+[ix]


X = torch.tensor(X)
Y = torch.tensor(Y)

machine learning engineer ii
... ---> m
..m ---> a
.ma ---> c
mac ---> h
ach ---> i
chi ---> n
hin ---> e
ine --->  
ne  ---> l
e l ---> e
 le ---> a
lea ---> r
ear ---> n
arn ---> i
rni ---> n
nin ---> g
ing --->  
ng  ---> e
g e ---> n
 en ---> g
eng ---> i
ngi ---> n
gin ---> e
ine ---> e
nee ---> r
eer --->  
er  ---> i
r i ---> i
 ii ---> .
senior machine learning engineer  ii
... ---> s
..s ---> e
.se ---> n
sen ---> i
eni ---> o
nio ---> r
ior --->  
or  ---> m
r m ---> a
 ma ---> c
mac ---> h
ach ---> i
chi ---> n
hin ---> e
ine --->  
ne  ---> l
e l ---> e
 le ---> a
lea ---> r
ear ---> n
arn ---> i
rni ---> n
nin ---> g
ing --->  
ng  ---> e
g e ---> n
 en ---> g
eng ---> i
ngi ---> n
gin ---> e
ine ---> e
nee ---> r
eer --->  
er  --->  
r   ---> i
  i ---> i
 ii ---> .
senior machine learning engineer
... ---> s
..s ---> e
.se ---> n
sen ---> i
eni ---> o
nio ---> r
ior --->  
or  ---> m
r m ---> a
 ma ---> c
mac ---> h
ach ---> i
chi ---> n
hin ---> e
ine --->  
ne  ---> l

In [66]:
block_size = 3
X, Y = [], []
for w in job_names:
    # print(w)
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context),'--->',itos[ix])
        context = context[1:]+[ix]


X = torch.tensor(X)
Y = torch.tensor(Y)

In [67]:
X.shape, Y.shape

(torch.Size([25826, 3]), torch.Size([25826]))

In [77]:
g = torch.Generator().manual_seed(142)
C = torch.randn((28,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,28), generator=g)
b2 = torch.randn(28, generator=g)
parameters = [C, W1, b1, W2, b2]

In [78]:
sum([p.numel() for p in parameters])

3584

In [79]:
X.shape[0], prob.shape

(25826, torch.Size([25826, 27]))

In [80]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
loss = -prob[torch.arange(25826), Y].log().mean()
loss

tensor(17.4761)

In [81]:
F.cross_entropy(logits, Y)

tensor(17.4761)

In [82]:
for p in parameters:
    p.requires_grad = True

In [103]:
for i in range(100):
    emb = C[X]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    for p in parameters:
        p.grad = None
    
    loss.backward()
    for p in parameters:
        p.data += -0.1*p.grad
    if i%10 == 0:
        print(loss.item())

2.0488367080688477
2.022970199584961
1.9992038011550903
1.9772974252700806
1.9570364952087402
1.938232660293579
1.9207197427749634
1.9043480157852173
1.888980746269226
1.8744957447052002


In [126]:
g = torch.Generator().manual_seed(100)
for _ in range(20):
    out = []
    context = [0]* block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator = g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break

    print(''.join(itos[i] for i in out))

sentir.
saeaialest.
csenior mne.
rie.
seni recsengirdarakoiengineert.
siniagalpiatnci ng eagscieniorlsengir  aepiatesti.
sagrrcsaeesoprsentister mneagadadedeo dng engine pile aianaen ing enieraphmachineer.
csenior.
eadeciencipnthrretntngn aen  eaagaea oa schrnei dic  attkt.
ses.
aeninginier asaiese.
tie liefe.
sad l  s eecse  calcts iec in  ee r l lagdlo ri asciencigr  s dec mngenior medltmader dcncbaniagid sne chiintancigneachmatni r aaveejafoagnd ltiaaajctenciencepe nti de.
tie.
senfealalngnelo.
aiie sotoat atain  englngsne ntientwarnginaening rtcseniniatnini lealiarn in  entist.
siie soa scineer ml  ageie.
l ineer adaiadat ascientisi.
ltst ainagndagachineer.
seneiafiurve.


In [106]:
model = nn.Sequential(
    nn.Linear(in_features = 28, out_features = 6, bias=False),
    nn.Linear(in_features = 6, out_features = 100),
    nn.Tanh(),
    nn.Linear(in_features = 100, out_features = 28),
    nn.Softmax()
)
model

Sequential(
  (0): Linear(in_features=28, out_features=6, bias=False)
  (1): Linear(in_features=6, out_features=100, bias=True)
  (2): Tanh()
  (3): Linear(in_features=100, out_features=28, bias=True)
  (4): Softmax(dim=None)
)

In [128]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr = 0.1)

In [ ]:
for epoch in range(100):
    model.train()
    pred = model(X)
    y_logits = model(self.X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
    y_pred = nn.functional.softmax(y_logits) # turn logits -> pred probs -> pred labls
    
    loss = loss_fn(y_logits, Y)
    
    optimizer.zero_grad()
    
    loss.backward()

    optimizer.step()

    